<a href="https://colab.research.google.com/github/keremjn4/repo/blob/main/text_based_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from os import name
import datetime
import time
import threading
import random

##keys needs to be reconsidered.
##story of game needs to be advanced (scenario & implementation to game mechanics)
##talking with npc's needs to be added
##travel mechanics & switching rooms & opening doors
##rooms and items needs to be implemented to json
##game map


class Player:
    def __init__(self, name, equipped_items, position):
        """Initialize the player's name, health, and inventory."""
        self.name = name
        self.health = 100
        self.inventory = []
        self.equipped_items = equipped_items
        self.alive = True
        self.basedamage = 3
        self.damage = 3
        self.magic = 0
        self.armor = 0
        self.keys = []
        self.position = position
        self.critical_chance_multiplier = 1
        self.critical_chance = 0.01*self.critical_chance_multiplier
        self.critical_damage_multiplier = random.randint(2,3)
        self.incombat = False

    def update_damage(self):
        self.damage = self.basedamage + sum(item.damage for item in self.equipped_items)
        self.magic = sum(item.magic for item in self.equipped_items)
        self.armor = sum(item.armor for item in self.equipped_items)

    def search(self, environment,item):
        """Search the current room for items."""
        environment_items_names = ", ".join([item.name for item in environment.items])
        if environment.items:
           print(f"You see the following items: {environment_items_names}")
        else:
           print("You don't see anything worthy")

    def add_to_inventory(self, environment, item):
        if not environment.items:
            print("There are no items to pick up.")
            return
        """Add an item to the player's inventory."""
        item_to_take = input("Which item you wan't to take?").lower()
        item_to_take_list = [item.strip() for item in item_to_take.split(",")]

        for i in item_to_take_list:
            item_actual = next((item for item in environment.items if item.name == i), None)
            if len(self.inventory) >= 4 and item_actual.id !=1:
                print("Your hands are full. You can't hold more items.")
                break

            elif item_actual in environment.items and item_actual.id != 1:
                self.inventory.append(item_actual)
                environment.items.remove(item_actual)
                print(f"You picked up {item_actual.name}.")
            elif item_actual in environment.items and item_actual.id == 1:
                 self.inventory.append(item_actual)
                 environment.items.remove(item_actual)
                 print(f"You picked up {item_actual.name}.")
            else:
                print(f"You can't take {i}.")

    def drop_item(self, environment, item):
        """Drop an item from the player's inventory."""
        item_to_drop = input("which items you want to drop?").lower()
        item_to_drop_list = item_to_drop.split(",")
        for i in item_to_drop_list:
            actual_obj = next((item for item in self.inventory if item.name == i), None)
            if actual_obj:
                self.inventory.remove(actual_obj)
                environment.items.append(actual_obj)
                if actual_obj.equipped:
                 self.equipped_items.remove(actual_obj)
                 environment.items.append(actual_obj)
                print(f"\n{actual_obj.name} has removed from your inventory.")
            else:
                print(f"\nYou don't have {i} in your inventory.")

    def show_inventory(self, item):
        if not self.inventory:
            print("Your inventory is empty.")
        else:
            if self.equipped_items:
                equipped_items_names = ", ".join([item.name for item in self.equipped_items])
                print(f"\nYour equipped items: {equipped_items_names}")
            elif not self.equipped_items:
                print("\nyou don't have any equipped items")
            inventory_items_names = ", ".join([item.name for item in self.inventory])
            print(f"\nYour inventory: {inventory_items_names}")

    def equip(self, environment, item):
        action = input("\nwhich item you wan't to equip?").lower()
        item_to_equip_list = [item.strip() for item in action.split(",")]
        for i in item_to_equip_list:
          action_obj = next((item for item in self.inventory if item.name == i), 0)
          if action_obj == 0:
            print(f"{action} is not in your inventory")
          else:
            if action_obj.equipped:
               print(f"{action_obj.name} is already equipped")
               continue
            self.equipped_items.append(action_obj)
            action_obj.equipped = True
            self.update_damage()
            print(f"{action_obj.name} is equipped")

    def unequip(self, item):
         action = input("\nwhich item you wan't to unequip?").lower()
         item_to_unequip_list = [item.strip() for item in action.split(",")]
         for i in item_to_unequip_list:
            action_obj = next((item for item in self.equipped_items if item.name == i), 0)
            if action_obj == 0:
               print(f"{action} is not equipped")
            else:
               self.equipped_items.remove(action_obj)
               action_obj.equipped = False
               self.update_damage()
               print(f"{action_obj.name} is unequipped")

    def combat(self, enemy, environment):
      while self.alive and enemy.alive:
        print(f"\n {self.name}'s Health: {self.health}")
        print(f"\n {enemy.name}'s Health: {enemy.health}")
        action = input("Choose an action: ").lower()
        if action == "attack":
            player_critical = random.random() <= self.critical_chance
            enemy_evade = random.random() <= enemy.evade_chance
            if enemy_evade:
               print(f"{enemy.name} evaded the attack!")
               enemy.enemy_attack(self)
               continue
            if player_critical:
               enemy.health -= self.critical_damage_multiplier*self.damage
               print(f"\n {self.name} executed CRITICAL HIT! {self.critical_damage_multiplier*self.damage} damage.")
            elif not player_critical:
              enemy.health -= self.damage
              print(f"\n {self.name} attacks {enemy.name} for {self.damage} damage!")
            if enemy.health <= 0:
              enemy.alive = False
              environment.enemies.remove(enemy)
              environment.enemy_list.remove(enemy.name)
              break
            if enemy.alive:
              enemy.enemy_attack(self)
        elif action == "flee":
             print(f"{self.name} flees from {enemy.name}")
             break
      if not self.alive:
         print(f"\n{self.name} has been defeated!")
      elif not enemy.alive:
         print(f"\n{enemy.name} has been defeated!")


    def interactions(self, environment):
        while True:
            # Prompt the player to choose an NPC to interact with
            choose_npc = input(f"Who do you want to interact with? Available: {environment.npc_list}\n {environment.enemy_list}").lower()

            if choose_npc in environment.npc_list:
                # If NPC exists, ask what the player wants to do
                player_interaction_decision = input(f"What do you want to do with {choose_npc}? ").lower()
                print(f"You chose to {player_interaction_decision} with {choose_npc}.")
                # Implement your interaction logic here
            elif choose_npc in environment.enemy_list:
                 enemy = next((enemy for enemy in environment.enemies if enemy.name.lower() == choose_npc), None)
                 if enemy and enemy.alive:
                    self.combat(enemy, environment)
                    break
                 else:
                    print(f"{choose_npc} is not here.")
            else:
                # If NPC does not exist, ask if they want to try again
                choose_another_npc = input(f"\n{choose_npc} is not here. Do you want to try someone else? (yes/no): ").lower()
                if choose_another_npc == "no":
                    print("\nYou decided not to interact with anyone.")
                    break
                elif choose_another_npc == "yes":
                    continue  #!!!!!!!!!Restart the loop to choose another NPC!!!!!!!!!!
                else:
                    print("Invalid input.")
                    break

    def switch_room(self,environment_list):
        environment_list_names = ",\n".join([environment.name for environment in environment_list])
        travel = input(f"\nWhere do you wan't to go?\n {environment_list_names}\n")
        if travel in [environment.name for environment in environment_list]:
            selected_environment = next((environment for environment in environment_list if environment.name == travel), "This Place Does not exist")
            self.position = selected_environment
            npc_names = ", ".join([selected_environment.npc for npc in selected_environment.npc_list])
            print(selected_environment.description)
            print(f"\ncharacters: {npc_names}")
            if selected_environment.enemies:
                enemy_names = ", ".join([enemy.name for enemy in selected_environment.enemies])
                print(f"\n enemies: {enemy_names}")


    def list_of_actions(self):
       actions = ["search", "take", "inventory", "calendar", "drop", "interact", "equip", "attack", "flee", "map"]
       print(f"available actions: {actions}")
       return actions

class Enemy:
    def __init__(self, name, health, damage, _type):
        self.name = name
        self.health = health
        self.damage = damage
        self.alive = True
        self._type = _type
        self.evade_chance = 0.1
    def enemy_attack(self, player):
       if self.alive:
        player.health -= self.damage
        print(f"\n {self.name} attacks {player.name} for {self.damage} damage!")
        if player.health <= 0:
            player.health = 0
            player.alive = False
    def enemy_take_damage(self, player):
       if self.alive:
        self.health -= player.damage
        print(f"\n {self.name} takes {player.damage} damage!")
        if self.health <= 0:
            self.health = 0
            self.alive = False


class Environment:
    description_of_roomA = ("the holy yeghegetsi. or what is left from this sacred site... \n crossing the narthex, felt the strong scent of burned flesh. \n you see a vartabed is looking at the ashes and the decayed corpses while he strokes his long, black beard..\n hood of his black robe covers his eyes. you can't read his emotions.")
    def __init__(self, name, description, items, npc=None, enemies=None):
        """Initialize a room with a description, items, and an optional NPC."""
        self.name = name
        self.description = description
        self.npc = npc
        self.items = items
        #print(f"\n You have entered {self.name}.")
        self.npc_list = self.npc.split(",")
        #print(f"\n people in the room: {npc_names}")
        self.enemies = enemies or []
        self.enemy_list = [enemy.name for enemy in self.enemies]
            #print(f"\nEnemies in the room: {enemy_names}")



class item:
    def __init__(self, name, damage, armor, magic, description, id):
     self.name = name
     self.damage = damage
     self.armor = armor
     self.magic = magic
     self.description = description
     self.equipped = False
     self.id = id

class Game_Calendar:
    def __init__(self):
        self.months =  [{"name": "har", "days": 31, "monthno":1},
                        {"name": "safer", "days": 29,"monthno":2},
                        {"name": "rebiulevvel", "days": 31,"monthno":3},
                        {"name": "rebulahr", "days": 30,"monthno":4},
                        {"name": "cemaz", "days": 31,"monthno":5},
                        {"name": "cemazlahr", "days": 30,"monthno":6},
                        {"name": "rajap", "days": 31,"monthno":7},
                        {"name": "sabn", "days": 31,"monthno":8},
                        {"name": "nazamar", "days": 30,"monthno":9},
                        {"name": "sevbas", "days": 31,"monthno":10},
                        {"name": "zilkade", "days": 30,"monthno":11},
                        {"name": "zilhice", "days": 31,"monthno":12}]

        self.weekdays =  [{"name": "seme", "day":1},
                         {"name": "yewseme", "day":2},
                         {"name": "diseme", "day":3},
                         {"name": "seseme", "day":4},
                         {"name": "chʿorek", "day":5},
                         {"name": "hinkshaptʿi", "day":6},
                         {"name": "owrpat", "day":7}]

        self.current_year = 154
        self.current_month = self.months[0]
        self.current_day = self.weekdays[0]
        self.current_month_day = 1
        current_month_name = self.current_month["name"]
        current_day_name = self.current_day["name"]
        self.running = True
        self.start_time = time.time()
        self.sunpos = time.time()
        print(f"today is {self.current_month_day} / {current_month_name} / {self.current_year}, {current_day_name}")

    def time_advancer(self):
        time_elapsed = time.time() - self.start_time
        sun_position = time.time() - self.sunpos
        if sun_position >= 100:
            print("\nthe sun has set and the moon has risen")
            self.sunpos = time.time()
            """Advance the time by one day."""
        if time_elapsed >= 200:  # 200 seconds simulate one day
            self.current_day = self.weekdays[self.current_day["day"] -1 + 1]
            self.current_month_day += 1
            self.start_time = time.time()
            print(f"\nsun rises. a new day has started:")
            if self.current_day["day"] == 7:
                self.current_day = self.weekdays[0]
            if self.current_month_day > self.current_month["days"]:
                self.current_month = self.months[self.current_month["monthno"] -1 + 1]
                self.current_month_day = 1
            if self.current_month["monthno"] == 12:
                self.current_year += 1
                self.current_month = self.months[0]

    def player_calendar(self):
       print(f"{self.current_month_day} / {self.current_month['name']} / {self.current_year}, {self.current_day['name']}")

def main():
    key =  item(name= "key", damage = 0, armor = 0, magic = 0, description = "key", id = 1)
    book = item(name ="book", damage = 0, armor = 0, magic = 0, description = "a book about history", id = 2)
    sword = item(name = "sword", damage = 20, armor = 0, magic= 0, description = "common sword", id =  3)
    player_name = input("Enter your player's name: ")
    calendar = Game_Calendar()
    monster = [Enemy("monster", 100, 20, "common"),Enemy("monster", 100, 20,"common")]
    roomA = Environment(name = "yeghegetsi", description=Environment.description_of_roomA, items=[sword, book, key], npc="vartabed", enemies= monster)
    roomB = Environment(name = "test", description="This is another room", items=[sword, book], npc="mairig", enemies= monster)
    environment_list = [roomA,roomB]
    player = Player(player_name,equipped_items=[],position=roomA)
    room_file = open('/content/rooms.json',"r").read()


    while True:

        if not player.alive:
           print("game over")
           break

        calendar.time_advancer()
        print("\nWhat do you want to do?")
        action = input("Choose an action: ").lower()


        if action == "search":
            player.search(player.position,item)

        elif action == "take":
              player.add_to_inventory(player.position,item)

        elif action == "inventory":
              player.show_inventory(item)

        elif action == "drop":
              player.drop_item(player.position,item)

        elif action == "interact":
              player.interactions(player.position)

        elif action == "calendar":
              calendar.player_calendar()

        elif action == "help":
            player.list_of_actions()

        elif action == "equip":
            player.equip(player.position,item)

        elif action == "unequip":
            player.unequip(item)

        elif action == "map":
             player.switch_room(environment_list)

        elif action == "e":
            print("Exiting the game. Goodbye!")
            break

        else:
            print("you can't do that")

if __name__ == "__main__":
    main()


Enter your player's name: kerem
today is 1 / har / 154, seme

What do you want to do?
Choose an action: e
Exiting the game. Goodbye!


In [10]:
# Create and save a JSON file
import json

room_data = {
    "rooms": [
        {
            "name": "yeghegetsi",
            "description": "the holy yeghegetsi...",
            "items": ["key", "book", "sword"],
            "npc": "vartabed",
            "enemies": ["monster"],
            "room_id": 0
        },
        {
            "name": "test",
            "description": "This is another room.",
            "items": ["sword", "book"],
            "npc": "mairig",
            "enemies": ["monster"],
            "room_id": 1
        }
    ]
}

item_data = {
    "items": [
        {
            "name": "key",
            "damage": 0,
            "armor": 0,
            "magic": 0,
            "description": "key",
            "item_id": 0

        },
        {
            "name": "book",
            "damage": 0,
            "armor": 0,
            "magic": 0,
            "description": "a book about history",
            "item_id": 1
        },
        {
            "name": "sword",
            "damage": 20,
            "armor": 0,
            "magic": 0,
            "description": "common sword",
            "item_id": 2
        }
    ]
}


# Save it to a JSON file
with open('rooms.json', 'w') as json_file:
    json.dump(room_data, json_file, indent=4)
print("JSON file created!")
with open('items.json', 'w') as json_file:
    json.dump(item_data, json_file, indent=4)
print("JSON file created!")


JSON file created!
JSON file created!


In [ ]:

class Rooms_Json:
      def __init__(self):
       path = '/content/rooms.json'
       with open(path,"r") as rooms_file:
        self.rooms = json.load(rooms_file)["rooms"]
        self.room = self.rooms[0]
       self.rooms_list = [room for room in self.rooms]


      def print_rooms(self):
        #print(f"{self.rooms_list}")
        print(f"{self.room['items']}")



rooms_instance = Rooms_Json()
rooms_instance.print_rooms()









['key', 'book', 'sword']


In [18]:
import json

class generateitem:
    def __init__(self,**kwargs):
      for key, value in kwargs.items():
          setattr(self, key, value)
      ##need to read existing file, convert to a list, append it with new item, write it back again.
      ##need to write inside rooms.json
    def to_json(self,path = None):
      #  with open(path, "r") as file:
          #   data = json.load(file)
      #  data["items"].append(self.__dict__)
        with open('/content/rooms.json',"r") as file:
             data = json.load(file)
        for room in data['rooms']:
          if room['room_id'] == self.item_room_id:
             room['items'].append(self.__dict__)

        with open(path,'w') as file:
            json.dump(data,file,indent=4)

        print("new content has added")

class generateenemy:
    def __init__(self,**kwargs):
      for key, value in kwargs.items():
          setattr(self, key, value)
      ##need to read existing file, convert to a list, append it with new item, write it back again.
      ##need to write inside rooms.json
    def to_json(self,path = None):
      #  with open(path, "r") as file:
          #   data = json.load(file)
      #  data["items"].append(self.__dict__)
        with open('/content/rooms.json',"r") as file:
             data = json.load(file)
        for room in data['rooms']:
          if room['room_id'] == self.enemy_room_id:
             room['enemies'].append(self.__dict__)

        with open(path,'w') as file:
            json.dump(data,file,indent=4)

        print("new content has added")


#item = generateitem(name="ring", damage=5, armor=0, magic=0, description="gold ring", item_room_id = 3)
#item.to_json('/content/rooms.json')

enemy = generateenemy(name="abomination", health = 100, damage = 10, _type = "common", enemy_room_id = 3)
enemy.to_json('/content/rooms.json')








new content has added


In [15]:
class generateenv:
   def __init__(self,**kwargs):
      for key, value in kwargs.items():
          setattr(self, key, value)
      ##need to read existing file, convert to a list, append it with new item, write it back again.
   def to_json(self,path = None):
        with open(path, "r") as file:
             data = json.load(file)
        data["rooms"].append(self.__dict__)


        with open(path,'w') as file:
            json.dump(data,file,indent=4)

        print("new content has added")




street = generateenv(name="street", description="a narrow street", items=[], npc=[], enemies=[], room_id = 3)
street.to_json('/content/rooms.json')

new content has added


In [5]:
with open('/content/rooms.json',"r") as file:
 room_data = json.load(file)
 print(room_data)

for room in room_data['rooms']:
  print(room['name'])
  print(room['items'])


{'rooms': [{'name': 'yeghegetsi', 'description': 'the holy yeghegetsi...', 'items': ['key', 'book', 'sword'], 'npc': 'vartabed', 'enemies': ['monster']}, {'name': 'test', 'description': 'This is another room.', 'items': ['sword', 'book'], 'npc': 'mairig', 'enemies': ['monster']}]}
yeghegetsi
['key', 'book', 'sword']
test
['sword', 'book']
